# Aufgabe 2.3
**Thema: Lineare Regression auf Airfoil-Noise-Daten**<br>
Betrachten Sie nun die Excel-Datensammlung airfoil_self_noise.xls (siehe Vorlesungsverzeichnis): Sie besteht aus N = 1503 Datensätzen der Dimension D = 6, welche den Schalldruck (Spalte 6) einer Flugzeugtragfläche in Abhängigkeit verschiedener Parameter (wie z. B. Anströmwinkel und -geschwindigkeit; Spalten 1-5) darstellen. Sie sollen auf die-
sen Daten ein Regressionsmodell lernen, um für neue Parametersätze den resultierenden Schalldruck vorhersagen zu können.

## a)
Vervollständigen Sie das Programmgerüst V2A3_regression_airfoilnoise.py um eine Least-Squares-Regression auf den Daten zu berechnen. Optimieren Sie die Hyper-Parameter um bei einer S = 3-fachen Kreuzvalidierung möglichst kleine Fehlerwerte zu erhalten.
   - Welche Bedeutung haben jeweils die Hyper-Parameter lmbda, deg, flagSTD?
       - lmbda: Regularisierungs-Koeffizient
       - deg: Grad der Basisfunktion
       - flagSTD: falls >0 werden die Daten standardisiert
   - Was passiert ohne Skalierung der Daten (flagSTD=0) bei höheren Polynomgraden (achten Sie auf die Werte von maxZ)?
       - EXCEPTION DUE TO BAD CONDITION:flagOK= 0  maxZ= 2.1992434255935223e-06 <br> der Fehlerwert geht hoch ohne die Sḱalierung
   - Geben Sie Ihre optimalen Hyper-Parameter sowie die resultierenden Fehler-Werte an.
       - lmbda = 4, deg = 5, flagSTD = 1
       - mean = 0.017310251510489143
   - Welche Prognosen ergibt Ihr Modell für die neuen Datenvektoren x_test_1=[1250,11,0.2,69.2,0.0051] bzw. x_test_2=[1305,8,0.1,57.7,0.0048]?
       - y(x_test_1) = 130.54643295319002
       - y(x_test_2) = 132.04456194433678
   - Welchen Polynomgrad und wieviele Basisfunktionen verwendet Ihr Modell?
       - Grad 5
       - 252 Basisfunktionen


## b)
Vervollständigen Sie das Programmgerüst V2A3_regression_airfoilnoise.py um eine KNN-Regression auf den Daten zu berechnen. Optimieren Sie die Hyper-Parameter um bei einer S = 3-fachen Kreuzvalidierung möglichst kleine Fehlerwerte zu erhalten.
   - Welche Bedeutung haben jeweils die Hyper-Parameter K und flagKLinReg?
       - K: Anzahl der Nearest Neigbors die mit einbezogen werden
       - flagKLinReg: falls >0 wird und K>D wird lineare Regression verwendet
   - Geben Sie Ihre optimalen Hyper-Parameter sowie die resultierenden Fehler-Werte an.
       - lmbda = 4, K = 20, flagKLinReg = 1, deg = 5, flagSTD = 1
       - mean = 0.02379121048873416
   - Welche Prognosen ergibt Ihr Modell für die neuen Datenvektoren x_test_1=[1250,11,0.2,69.2,0.0051] bzw. x_test_2=[1305,8,0.1,57.7,0.0048]?
       - y(x_test_1) = -1937.5523974696957
       - y(x_test_2) = 132.04456194433678

## c)
Vergleichen Sie die beiden Modelle. Welches liefert die besseren Ergebnisse?

Die Least Squares Methode liefert bessere Ergebnisse. <br>
KNN liefert ohne und mit dem Flag für lineare Regression schlechtere Ergebnisse.
Mit Flag läuft der KNN ebenfalls mit Least Squares.

In [2]:
# to log outputs start with: python V2A3_regression_airfoilnoise.py >V2A3_regression_airfoilnoise.log

import numpy as np
import pandas as pd

from V2A2_Regression import *

In [11]:
# ***** MAIN PROGRAM ********
# (I) Hyper-Parameters
S=3;               # S-fold cross-validation
lmbda=4;           # regularization parameter (lambda>0 avoids also singularities)
K=2;               # K for K-Nearest Neighbors
flagKLinReg = 0;   # if flag==1 and K>=D then do a linear regression of the KNNs to make prediction
deg=5;             # degree of basis function polynomials 
flagSTD=1;         # if >0 then standardize data before training (i.e., scale X to mean value 0 and standard deviation 1)
N_pred=5;          # number of predictions on the training set for testing
x_test_1 = [1250,11,0.2,69.2,0.0051];   # REPLACE dummy code: define test vector 1
x_test_2 = [1305,8,0.1,57.7,0.0048];   # REPLACE dummy code: define test vector 2

In [9]:
# (II) Load data 
fname='../../DATA/AirfoilSelfNoise/airfoil_self_noise.xls'
airfoil_data = pd.read_excel(fname,0); # load data as pandas data frame 
T = airfoil_data.values[:,5]           # target values = noise load (= column 5 of data table)
X = airfoil_data.values[:,:5]          # feature vectors (= column 0-4 of data table)
N,D=X.shape                            # size and dimensionality of data set
idx_perm = np.random.permutation(N)    # get random permutation for selection of test vectors 
print("Data set ",fname," has size N=", N, " and dimensionality D=",D)
print("X=",X)
print("T=",T)
print("x_test_1=",x_test_1)
print("x_test_2=",x_test_2)
print("number of basis functions M=", len(phi_polynomial(X[1],deg)))

Data set  ../../DATA/AirfoilSelfNoise/airfoil_self_noise.xls  has size N= 1502  and dimensionality D= 5
X= [[1000.            0.            0.3048       71.3           0.00266337]
 [1250.            0.            0.3048       71.3           0.00266337]
 [1600.            0.            0.3048       71.3           0.00266337]
 ...
 [4000.           15.6           0.1016       39.6           0.0528487 ]
 [5000.           15.6           0.1016       39.6           0.0528487 ]
 [6300.           15.6           0.1016       39.6           0.0528487 ]]
T= [125.201 125.951 127.591 ... 106.604 106.224 104.204]
x_test_1= [1250, 11, 0.2, 69.2, 0.0051]
x_test_2= [1305, 8, 0.1, 57.7, 0.0048]
number of basis functions M= 252


In [5]:
# (III) Do least-squares regression with regularization 
print("\n#### Least Squares Regression with regularization lambda=", lmbda, " ####")
#lsr = None  # REPLACE dummy code: Create and fit Least-Squares Regressifier using polynomial basis function of degree deg and flagSTD for standardization of data
phi=lambda x: phi_polynomial(x,deg)
lsr = LSRRegressifier(lmbda, phi, flagSTD)
lsr.fit(X,T)
print("lsr.W_LSR=",lsr.W_LSR)    # REPLACE dummy code: print weight vector for least squares regression  
print("III.1) Some predictions on the training data:")
for i in range(N_pred): 
    n=idx_perm[i]
    print("Prediction for X[",n,"]=",X[n]," is y=",lsr.predict(X[n]),", whereas true value is T[",n,"]=",T[n])   # REPLACE dummy code: compute prediction for X[n]
print("III.2) Some predicitions for new test vectors:")
print("Prediction for x_test_1 is y=", lsr.predict(x_test_1))    # REPLACE dummy code: compute prediction for x_test_1
print("Prediction for x_test_2 is y=", lsr.predict(x_test_2))    # REPLACE dummy code: compute prediction for x_test_2
print("III.3) S=",S,"fold Cross Validation:")
err_abs,err_rel = lsr.crossvalidate(S,X,T)                 # REPLACE dummy code: do cross validation!! 
print("absolute errors (E,sd,min,max)=", err_abs, "\nrelative errors (E,sd,min,max)=", err_rel)


#### Least Squares Regression with regularization lambda= 4  ####


C:\Users\bjoer\Documents\Praktikum\Aufgabe 2\Abgabe\V2A2_Regression.py:322: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  self.W_LSR = np.linalg.lstsq(PHI.T.dot(PHI) + np.dot(lmbda,np.eye(self.M)), PHI.T.dot(T))[0]


lsr.W_LSR= [-0.29232085 -1.53595232 -0.31947921 -0.71582724  0.26783336 -0.49238806
  0.36593902 -0.55739161  0.10486601 -0.03292355  0.16969493  0.01386179
 -0.02385756  0.1213603   0.00986892  0.00270992 -0.0523817   0.02856123
 -0.12391539 -0.12540552 -0.00641665  0.21542228  0.18849836  0.73746304
 -0.05511848  0.73652013  0.16539471  0.0870961  -0.0868486   0.32536833
  0.16486434 -0.16239675  0.14711298 -0.24586486 -0.01045699  0.16436901
 -0.1041675  -0.13037834 -0.0666859  -0.09124863  0.01428912 -0.06590659
 -0.17289218 -0.11145656 -0.13902407 -0.12881238  0.18380523  0.06229881
  0.02452736 -0.09228024  0.00105904 -0.24518279  0.08586138 -0.02458379
 -0.11132804 -0.24116303 -0.14961373  0.13712723 -0.25624705  0.02217059
 -0.53911543 -0.1086547   0.45811671 -0.05603751 -0.11314852 -0.25850148
 -0.00481869 -0.07414863  0.02165336  0.05184426 -0.20601768  0.01537968
 -0.03815796  0.04678697 -0.06887814  0.00490531 -0.06447269 -0.09827021
  0.00393971  0.05471029 -0.18367597 -0.

In [12]:
# (IV) Do KNN regression  
print("\n#### KNN regression with flagKLinReg=", flagKLinReg, " ####")
knnr = KNNRegressifier(K,flagKLinReg)                                    # REPLACE dummy code: Create and fit KNNRegressifier
knnr.fit(X,T)
print("IV.1) Some predictions on the training data:")
for i in range(N_pred): 
    n=idx_perm[i]
    print("Prediction for X[",n,"]=",X[n]," is y=",knnr.predict(X[n]),", whereas true value is T[",n,"]=",T[n])   # REPLACE dummy code: compute prediction for X[n]
print("IV.2) Some predicitions for new test vectors:")
print("Prediction for x_test_1 is y=", knnr.predict(x_test_1))    # REPLACE dummy code: compute prediction for x_test_1
print("Prediction for x_test_2 is y=", knnr.predict(x_test_2))    # REPLACE dummy code: compute prediction for x_test_2
print("IV.3) S=",S,"fold Cross Validation:")
err_abs,err_rel = knnr.crossvalidate(S,X,T)                   # REPLACE dummy code: do cross validation!! 
print("absolute errors (E,sd,min,max)=", err_abs, "\nrelative errors (E,sd,min,max)=", err_rel )


#### KNN regression with flagKLinReg= 0  ####
IV.1) Some predictions on the training data:
Prediction for X[ 555 ]= [630.           5.4          0.1524      55.5          0.00433288]  is y= 132.415 , whereas true value is T[ 555 ]= 131.807
Prediction for X[ 458 ]= [2500.           0.           0.1524      71.3          0.0015988]  is y= 130.361 , whereas true value is T[ 458 ]= 128.127
Prediction for X[ 1358 ]= [2500.           6.7          0.1016      55.5          0.0052139]  is y= 121.6335 , whereas true value is T[ 1358 ]= 124.273
Prediction for X[ 904 ]= [400.         15.4         0.0508     39.6         0.0282593]  is y= 122.9185 , whereas true value is T[ 904 ]= 125.353
Prediction for X[ 302 ]= [1250.            4.            0.2286       71.3           0.00400603]  is y= 130.5565 , whereas true value is T[ 302 ]= 131.718
IV.2) Some predicitions for new test vectors:
Prediction for x_test_1 is y= 129.435
Prediction for x_test_2 is y= 128.988
IV.3) S= 3 fold Cross Validation:
ab